In [1]:
import anthropic
import os

path = '/work/bioinformatics/s229618/api_keys/claude_key.txt'

with open(path, 'r') as file:
    key = file.read()

client = anthropic.Anthropic(
    api_key=key,
)
message = client.messages.create(
    model="claude-3-opus-20240229", #replace this
    max_tokens=1000,
    temperature=0,
    system="Respond only in pirate speak.",
    messages=[
        {"role": "user", "content": "Hi Claude."}
    ]
)
print(message.content)

[TextBlock(text='Ahoy there, me hearty! What brings ye to these waters on this fine day?', type='text')]


In [2]:
transcript_id = '10_0991_331330'
import pandas as pd
import nltk
import json
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer

def read_transcript_from_id(transcript_id, chunk_num=1):

    path_to_data_folder = '/archive/shared/sim_center/shared/ameer/'
    # path_to_data_folder = '/archive/shared/sim_center/shared/annie/GPT4 3-chunk/'
    # lookinto this dictionary to find the path
    # can also manually create the path and it would be faster but not by much


    merged_lookup = pd.read_csv(path_to_data_folder + 'grade_lookupv5.csv')

    path = merged_lookup[merged_lookup.id == transcript_id].path.iloc[0]

    path = path[:-4] + '.json'

    # Opening JSON file
    f = open(path)

    # returns JSON object as 
    # a dictionary
    json_transcript = json.load(f)

    transcript = ' '

    lines = json_transcript
    
    if chunk_num == 1: 
        for line in lines:
            if line['text'] != '\n':
                tok_line = line['text'].split(' ')
                for i in range(len(tok_line)):
                    transcript += ' ' + tok_line[i]
    
    else:
        transcript_chunks = []
        # for each chunk
        for n in range(chunk_num):
            transcript = ''
            # get the relevant lines
            start = n*int(len(lines)/chunk_num)
            end = (n+1)*int(len(lines)/chunk_num)
            if n == chunk_num-1: end = len(lines)

            for line in lines[start: end]:
                if line['text'] != '\n':
                    tok_line = line['text'].split(' ')
                    for i in range(len(tok_line)):
                        transcript += ' ' + tok_line[i]
            #append to transcript
            transcript_chunks.append(transcript)
        
        transcript = transcript_chunks


    return transcript

In [3]:
read_transcript_from_id(transcript_id, chunk_num=2)

["  Learners, you may begin your patient station, remove your cover sheet, jot down any... Thank you. Come in.  Come on in, man.  Hello, Mr. Pimbleton. Hi, my name is Coleman. I'm a medical student at UT Southwestern. I am here to ask you some questions. I just want to know how you are feeling today.  I've been feeling sick to my stomach.  Now I throw one up, I feel not good.  Okay, when did this start?  It started getting nauseous.  I was feeling nausea.  I had a stomachache.  I felt nause up. I am not good.  Okay, when did this start? I started getting nauseous two days ago. Threw up the first  time yesterday after breakfast, about 20 minutes after I ate. Any other episodes of vomiting?  Yeah, I threw up three or four times yesterday and then after breakfast today about five  minutes after I ate.  Okay.  And what does your vomit look like? It's just small bits of undigested food, whatever I eat.  Any blood in it?  No.  No?  No  Bile?  No  Just the chunks of food?  No  Okay  Now this 

In [4]:
def summarize(transcript):
    summary = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=4096,
        temperature=0,
        system="Please summarize the following text in one paragraph. 100 words. Do not add any information that is not in the text.",
        messages=[
            {"role": "user", "content": transcript}
        ]
    )

    return summary
#print(message.content)

In [8]:
#summary.content[0].text

In [5]:
def diarize(summary, transcript):
    diarization = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=4096,
        temperature=0,
        system='''The following text represents an audio transcript between a medical student and a patient, with the summary included below.
                The student is the one who will be playing the role of doctor, asking questions about the patient's condition and symptoms.
                Please diarize the following transcript in order to indicate who speaks when, using the format in the following example, with the label "Student" for the medical student and "Patient" for the patient (not their names):

                Student: Hello, how are you today?

                Patient: I am feeling sick. 

                Student: Oh no, how long have you felt sick? 

                Patient: About two days. 

                Separate speaker turns by an extra new line, as above ('\n\n'). The lines do not always have to be alternating labels. There may be consecutive lines from one speaker. Do not use any speaker labels for the two participants other than "Student" and "Patient."

                The transcript may contain some instructions for the student, coming from a third-party speaker. Please label this segment "Instructions: ". 
                If there are no instructions in the transcript, disregard this label. 

                IMPORTANT: Do not remove or add any words to the transcript other than the speaker labels. Also, do not add or remove any punctuation or change any spellings.''',
        messages=[
            {"role": "user", "content": 'summary: ' + summary.content[0].text + '\n\ntranscript: ' + transcript}
        ]
    )

    return diarization
#print(diarization.content)

In [10]:
#print(diarization.content[0].text)

In [6]:
def save_file(diarization, transcript_id, path = '/archive/shared/sim_center/shared/annie/claude-opus/'):
    #path = '/archive/shared/sim_center/shared/annie/claude-opus/'
    with open(path + transcript_id + ".txt", "w") as outfile:
        outfile.write('ID: ' + transcript_id + '\n\n' + diarization)

In [7]:
def claude_diarize(transcript_id):
    print(transcript_id)
    transcript = read_transcript_from_id(transcript_id)
    summary = summarize(transcript)
    diarization = diarize(summary, transcript).content[0].text
    save_file(diarization, transcript_id)

In [8]:
def claude_diarize_twochunk(transcript_id):

    print(transcript_id)
    transcript = read_transcript_from_id(transcript_id, chunk_num=2)

    # first chunk
    summary0 = summarize(transcript[0])
    diarization0 = diarize(summary0, transcript[0]).content[0].text

    # second chunk
    summary1 = summarize(transcript[1])
    diarization1 = diarize(summary1, transcript[1]).content[0].text

    diarization = diarization0 + '\n\n' + diarization1

    save_file(diarization, transcript_id)

In [9]:
read_transcript_from_id('02_0036_174595', chunk_num=2)

["  Thank you. I'm sorry. Are you Mr. Daniel? Yes. Hi, Mr Daniel. My name's Kelsey. I am a medical student here in the clinic. Kelsey, yes.  Kelsey? Kelsey yes, it's right there. Okay. If it is okay with you, I would like to just ask you a few questions about what brought you in today and then do a physical exam. Sounds good. Okay, so what brought you to the clinic today? I had ECIs and a runny nose.  ECI and run a running nose, okay.  Okay.  And when did that start?  About two months ago.  Two months.  Two weeks ago?  Was there anything different about, like, the kind of work you were doing?  No, it usually comes around this time every year. You can see the fall,  because I've had it for about five years. So where it comes in the Fall, but it's gotten worse the  past two weeks. Okay, I don't know. For the past five  years, every Fall you get this, these itchy eyes, this runny nose.  You know that it is coming and then it has gotten worst in  the last two weeks?  Correct.  Okay, can yo

In [10]:
id_set1 = ['01_0542_298135',
'02_0036_174595',
'03_0028_174553',
'04_0043_174686',
'05_0033_174804',
'06_0079_175106',
'07_0068_174641',
'08_0029_174576',
'09_0029_174582',
'10_0991_331330']

id_set2 = ['01_1080_366142',
           '02_1056_380177',
           '03_1500_380168',
           '04_1512_380182',
           '05_1066_380195',
           '06_1048_365209',
           '07_1111_380134',
           '08_1044_380133',
           '09_1039_380193',
           '10_1005_331402'
]

In [11]:
# temp for formatting
def remove_extraspace(path, ids):
    for id in ids: 
        this_path = path + id + '.txt'
        with open(this_path, 'r') as file:
            transcript = file.read()
        sections = transcript.split('  ')
        transcript = ''
        for section in sections:
            transcript += ' ' + section
        with open(path + id + ".txt", "w") as outfile:
            outfile.write(transcript)

In [75]:
remove_extraspace('/archive/shared/sim_center/shared/annie/claude-opus/', id_set1)

In [64]:
for id in id_set1[2:]:
    claude_diarize_twochunk(id)

03_0028_174553
04_0043_174686
05_0033_174804
06_0079_175106
07_0068_174641
08_0029_174576
09_0029_174582
10_0991_331330


In [62]:
claude_diarize_twochunk('02_0036_174595')

02_0036_174595
0 143
143 286


In [12]:
for id in id_set2:
    claude_diarize_twochunk(id)
remove_extraspace('/archive/shared/sim_center/shared/annie/claude-opus/', id_set2)

01_1080_366142


02_1056_380177
03_1500_380168
04_1512_380182
05_1066_380195
06_1048_365209
07_1111_380134
08_1044_380133
09_1039_380193
10_1005_331402


In [28]:
#temp
ids = ['01_1080_366142']
def remove_second_id(ids):
    path = '/archive/shared/sim_center/shared/annie/claude-opus/'
    for id in ids: 
        this_path = path + id + '.txt'
        with open(this_path, 'r') as file:
            transcript = file.readlines()
        transcript = transcript[2:]
        out = ''
        for line in transcript:
            if line == '\n': continue
            out += line + '\n'
        with open(path + id + ".txt", "w") as outfile:
            outfile.write(out)
        

In [29]:
remove_second_id(id_set1[1:] + id_set2[2:])

In [27]:
id_set1[1:] + id_set2[2:]

['02_0036_174595',
 '03_0028_174553',
 '04_0043_174686',
 '05_0033_174804',
 '06_0079_175106',
 '07_0068_174641',
 '08_0029_174576',
 '09_0029_174582',
 '10_0991_331330',
 '03_1500_380168',
 '04_1512_380182',
 '05_1066_380195',
 '06_1048_365209',
 '07_1111_380134',
 '08_1044_380133',
 '09_1039_380193',
 '10_1005_331402']